# Fast.AI Computational Linear Algebra

### Lesson 3

In [14]:
# Imports
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import decomposition

In [5]:
%matplotlib inline
np.set_printoptions(suppress=True)

In [6]:
# Set up data from last lesson
# Get Newsgroups data from sklearn
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics','sci.space']
remove = ['headers','footers','quotes']
newsgroups_train = fetch_20newsgroups(subset='train', 
                                     categories=categories,
                                     remove=remove)
newsgroups_test = fetch_20newsgroups(subset='test', 
                                     categories=categories,
                                     remove=remove)

In [9]:
# Count non-stop words using a dense matrix (to make it easier for this exercise)
vectorizer = CountVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(newsgroups_train.data).todense()
vectors.shape

(2034, 26576)

### Truncated SVD

Get vectors corresponding to largest singular values rather than entire space

Dimensionality benefits of NMF while capturing more of original matrix

In [11]:
# A (m x n) => U (m x r) S (r x r) V^T (r x n)  r << m,n

### Randomized SVD

Speed-up benefits compared to normal SVD

#### Full SVD

In [12]:
%time U, s, Vh = linalg.svd(vectors, full_matrices=False)

Wall time: 15.6 s


In [13]:
print(U.shape, s.shape, Vh.shape)

(2034, 2034) (2034,) (2034, 26576)


#### Randomized SVD sklearn

In [16]:
%time u, s, v = decomposition.randomized_svd(vectors, 5)

Wall time: 3.66 s


In [17]:
print(u.shape, s.shape, v.shape)

(2034, 5) (5,) (5, 26576)


#### Randomized SVD using scipy linalg

In [22]:
# Find random matrix that approximates range of A
def randomized_range_finder(A, size, n_iter=5):
    Q = np.random.normal(size=(A.shape[1], size))  # Start with random matrix from normal distribution
    for i in range(n_iter):
        Q,_ = linalg.lu(A @ Q, permute_l=True)  # Get closer to A using LU Factorization
        Q,_ = linalg.lu(A.T @ Q, permute_l=True)
    Q,_ = linalg.qr(A @ Q, mode='economic')  # QR Factorization, Slower than LU but more accurate
    return Q

In [20]:
# oversamples prevents losing important components accuracy
def randomized_svd(M, n_components, n_oversamples=10, n_iter=4):
    n_random = n_components + n_oversamples  # Size of random approximate matrix
    Q = randomized_range_finder(M, n_random, n_iter)  # Get Q
    B = Q.T @ M  # Project into right dimensional space
    Uhat, s, V = linalg.svd(B, full_matrices=False)  # SVD
    del B
    U = Q @ Uhat  # Get orthonormal U
    return U[:, :n_components], s[:n_components], V[:n_components, :]  # Only want up to the n_components

In [24]:
%time u,s,v = randomized_svd(vectors, 5)

Wall time: 2.34 s


In [25]:
u.shape, s.shape, v.shape

((2034, 5), (5,), (5, 26576))